### Alexnet Implementation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from alexnet import AlexNet
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import ConcatDataset
import numpy as np

In [2]:
# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_augmented = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=(-20, 20), translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the entire dataset
dataset = ImageFolder('train_imgs', transform=transform)
dataset_augmented  = ImageFolder('train_imgs', transform=transform_augmented)

In [3]:
# Define the AlexNet model
model = AlexNet(num_classes=len(dataset.classes))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [4]:
# Define the number of folds for k-fold cross-validation
k_folds = 3

# Create KFold object to generate k-fold splits
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=1)

# Create empty lists to store the accuracies for each fold
fold_accuracies = []
batchsize = 128

In [5]:
# Loop through the k-fold splits
for fold, (train_idxs, val_idxs) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold+1}/{k_folds}')

    ### Commenting out for better runtime?
    #for fold_i, (train_idx_i, val_idx_i) in enumerate(kfold.split(dataset_augmented)):
    #    if fold == fold_i:
    #        train_augmented_idxs = train_idx_i
    #        val_augmented_idxs = val_idx_i
    #        continue
    
    # Create data loaders for training and validation sets
    train_dataset = torch.utils.data.Subset(dataset, train_idxs)
    #train_augmented_dataset = torch.utils.data.Subset(dataset_augmented, train_augmented_idxs)
    #train_dataset = ConcatDataset([train_dataset, train_augmented_dataset])

    val_dataset = torch.utils.data.Subset(dataset, val_idxs)
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)


    # Train the model
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # Get the inputs and labels
            inputs, labels = data

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # Print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    # Evaluate the model on the validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy on validation set: %d %%' % (100 * correct / total))
    # Store the loss and accuracy for this fold
    fold_accuracies.append(100 * correct / total)

# Print the mean and standard deviation of the accuracies for all folds
print(f'Mean Accuracy: {np.mean(fold_accuracies):.4f}, Std Accuracy: {np.std(fold_accuracies):.4f}')

Fold 1/3
[1,   100] loss: 4.396
[2,   100] loss: 3.964
[3,   100] loss: 3.010
[4,   100] loss: 2.366
[5,   100] loss: 2.035
[6,   100] loss: 1.818
[7,   100] loss: 1.636
[8,   100] loss: 1.483
[9,   100] loss: 1.333
[10,   100] loss: 1.231
Accuracy on validation set: 58 %
Fold 2/3
[1,   100] loss: 1.316
[2,   100] loss: 1.156
[3,   100] loss: 1.050
[4,   100] loss: 0.977
[5,   100] loss: 0.887
[6,   100] loss: 0.777
[7,   100] loss: 0.726
[8,   100] loss: 0.667
[9,   100] loss: 0.611
[10,   100] loss: 0.531
Accuracy on validation set: 64 %
Fold 3/3
[1,   100] loss: 0.925
[2,   100] loss: 0.735
[3,   100] loss: 0.627
[4,   100] loss: 0.541
[5,   100] loss: 0.469
[6,   100] loss: 0.427
[7,   100] loss: 0.371
[8,   100] loss: 0.340
[9,   100] loss: 0.331
[10,   100] loss: 0.295
Accuracy on validation set: 74 %
Mean Accuracy: 65.8986, Std Accuracy: 6.8597


In [6]:
# Evaluate the model on the validation set
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on validation set: %d %%' % (100 * correct / total))
# Store the loss and accuracy for this fold
fold_accuracies.append(100 * correct / total)


Accuracy on validation set: 74 %


In [7]:
# Evaluate the model on the training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on training set: %d %%' % (100 * correct / total))

Accuracy on training set: 92 %


In [8]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    # Evaluate the model on the validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy on validation set: %d %%' % (100 * correct / total))

[1,   100] loss: 0.271


NameError: name 'test_loader' is not defined

In [ ]:
# Evaluate the model on the validation set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on validation set: %d %%' % (100 * correct / total))

Accuracy on validation set: 24 %
